In [1]:
import numpy as np
import scipy.linalg

Ref: Distributionally robust control of constrained stochastic systems

In [2]:
def dlqr(A,B,Q,R):
    """
    Calculates the optimal gain matrix K for system
    x[k+1] = A x[k] + B u[k]
    with cost = sum x[k].T*Q*x[k] + u[k].T*R*u[k], such that
    u[k] = -K x[k]
    
    Returns: 
        -K
    """
    # first, solve the ricatti equation
    P = np.matrix(scipy.linalg.solve_discrete_are(A, B, Q, R))
    # compute the LQR gain
    K = np.matrix(scipy.linalg.inv(B.T*P*B+R)*(B.T*P*A))
    return -K

In [91]:
def disc_linear_system(A,B,delta_t):
    '''
    Discrete a linear system with implicit Euler
    x[k+1] = (I - delta_t * A)^{-1} @ x[k] + (I - delta_t * A)^{-1} @ (delta_t * B) @ u[k]
    
    Returns:
        Ak
        Bk
    
    ''' 
    Nx = np.shape(A)[0]
    Ix = np.identity(Nx)
    
    Ak = np.linalg.inv(Ix - delta_t * A)
    Bk = np.linalg.inv(Ix - delta_t * A) @ (delta_t * B)
    
    
    return Ak, Bk

In [92]:
def RK4_np(f, x, u, t, h):
    """
    Runge-Kutta 4th order solver using numpy array data type.

    Args:
        f: A function returning first order ODE in 2D numpy array (Nx x 1).
        x: Current value (list or numpy array). 
        t: Current time.
        h: Step length.
    Returns:
        x_next: Vector of next value in 2D numpy array (Nx x 1)
    """
    x = np.reshape(x, (np.shape(x)[0], -1))    # Reshape x to col vector in np 2D array
    k1 = f(t, x, u)
    k2 = f(t + h / 2, x + h / 2 * k1, u)
    k3 = f(t + h / 2, x + h / 2 * k2, u)
    k4 = f(t + h, x + h * k3, u)
    x_next = x + h / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
    return x_next

In [93]:
def mass_string_ode(t, x, u):
    m = 2 #[kg]
    k1 = 3 # [N/m]
    k2 = 2 # [N/m]
    
    A = np.array([[0,1],[-k2/m, -k1/m]])
    B = np.array([[0],[1/m]])
    
    dot_x = A @ x + B @ u
    
    return dot_x

Continuous-time system

$\begin{aligned}\left[\begin{array}{c}\dot{x}_{1}(t) \\ \dot{x}_{2}(t)\end{array}\right] &=\left[\begin{array}{cc}0 & 1 \\ -k_{2} / m & -k_{1} / m\end{array}\right]\left[\begin{array}{l}x_{1}(t) \\ x_{2}(t)\end{array}\right]+\left[\begin{array}{c}0 \\ 1 / m\end{array}\right] u(t) \\ y(t) &=\left[\begin{array}{ll}1 & 0\end{array}\right]\left[\begin{array}{l}x_{1}(t) \\ x_{2}(t)\end{array}\right] \end{aligned}$

In [94]:
m = 2 #[kg]
k1 = 3 # [N/m]
k2 = 2 # [N/m]

para = [m, k1, k2]

In [95]:
A = np.array([[0,1],[-k2/m, -k1/m]])
B = np.array([[0],[1/m]])
delta_t = 0.1

In [96]:
Ak,Bk = disc_linear_system(A,B,delta_t)

In [103]:
Q = np.diag([10,1])
R = np.diag([1])

In [104]:
neg_K = dlqr(Ak,Bk,Q,R)

In [105]:
neg_K

matrix([[-1.64465128, -1.02359576]])

In [106]:
N_sim = 100

x_init = np.array([[2],[0]])
t0 = 0

xk = x_init
uk = 0
t = t0
h = delta_t

x_list = []
x_list += [xk.flatten().tolist()]
u_list = []
for i in range(N_sim):
    uk = neg_K @ xk
    u_list +=  uk.flatten().tolist() 
    x_kp1 = RK4_np(mass_string_ode, xk, uk, t, h)
    x_list += x_kp1.flatten().tolist()
    xk = x_kp1

In [107]:
u_list

[[-3.289302564456352],
 [-2.914942510901101],
 [-2.5598110182383618],
 [-2.2263935494494858],
 [-1.9163535315592184],
 [-1.6306546787177174],
 [-1.369672758698418],
 [-1.1332968450937406],
 [-0.9210202675311228],
 [-0.7320216075616044],
 [-0.5652361925567164],
 [-0.41941861782554035],
 [-0.2931968818221258],
 [-0.1851187540790109],
 [-0.09369101342841346],
 [-0.017412197937609175],
 [0.04520049970282325],
 [0.09558957459547412],
 [0.13514019742854322],
 [0.16516728509624534],
 [0.1869061255229036],
 [0.20150606471958082],
 [0.21002680236539475],
 [0.2134368811374406],
 [0.21261399383289373],
 [0.2083467703985401],
 [0.20133774379984534],
 [0.19220722883086472],
 [0.18149788119473242],
 [0.16967973526549884],
 [0.15715554774320428],
 [0.14426630086558034],
 [0.13129674292442273],
 [0.11848086557844487],
 [0.10600723691806505],
 [0.09402412650932193],
 [0.08264437383307388],
 [0.07194996476618828],
 [0.06199629215829093],
 [0.052816086281720964],
 [0.04442300911719047],
 [0.0368149132265

In [108]:
x_list

[[2, 0],
 [1.9826689238506545, -0.337883851188942],
 [1.9349862198417493, -0.6082152491804027],
 [1.8633130681783237, -0.8187867821451399],
 [1.7732520651954105, -0.9769752784440978],
 [1.669691328807902, -1.089693169266517],
 [1.5568526124067068, -1.163356605030776],
 [1.4383418800523347, -1.2038677983761676],
 [1.3172010373594736, -1.2166092885486843],
 [1.1959597318770925, -1.2064480423099309],
 [1.0766863343887605, -1.1777475217519346],
 [0.9610373890339324, -1.134386056870654],
 [0.8503049763687974, -1.0797800583539778],
 [0.7454615705111942, -1.0169107922407186],
 [0.6472020905436895, -0.9483536118440088],
 [0.5559829486632923, -0.8763087029491551],
 [0.4720579844946872, -0.8026325454778964],
 [0.3955112478724361, -0.7288694285283872],
 [0.3262866525834703, -0.6562824761564078],
 [0.2642145723515755, -0.5858837488486628],
 [0.20903548900109586, -0.5184630808900762],
 [0.16042083245903396, -0.454615397390631],
 [0.11799117417557238, -0.3947663273396302],
 [0.08133195071989681, -0.

In [112]:
!pip install cvxpy

In [111]:
!pip install --upgrade pip

     |████████████████████████████████| 1.5 MB 61 kB/s eta 0:00:013
  Attempting uninstall: pip
    Found existing installation: pip 20.3.3
    Uninstalling pip-20.3.3:
      Successfully uninstalled pip-20.3.3


In [113]:
!pip install pytest


In [114]:
!pytest cvxpy/tests

============================= test session starts ==============================
platform darwin -- Python 3.7.7, pytest-6.2.1, py-1.10.0, pluggy-0.13.1
rootdir: /Users/zhengangzhong/Dropbox/code/py/optimization/distributionally_robust_optimization
plugins: anyio-2.0.2
collected 0 items                                                              

============================ no tests ran in 0.00s =============================
ERROR: file or directory not found: cvxpy/tests

